In [2]:
# Ввод необходимых библиотек
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import enum 
import scipy.optimize as optimize
import datetime
import statsmodels.api as sm
import pandas as pd
#!pip install yahoo_fin
#from yahoo_fin import options
import time
import pickle
from scipy.interpolate import CubicSpline
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Вводим отображения чисел при выводе
np.set_printoptions(precision = 3)
# Вводим мнимую единицу i
i   = np.complex(0.0,1.0)

# Вводим класс отражающий тип опциона

class OptionType(enum.Enum):
    CALL = 1.0
    PUT = -1.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [ ]:
def CallPutOptionPriceCOSMthd(par, CP, S0, r, tau, K, N, L):
    # par  - параметры модели
    # CP   - тип опциона
    # S0   - начальная цена базового актива
    # r    - процентная ставка (постоянная)
    # tau  - время до экспирации
    # K    - лист страйков
    # N    - число элементов в разложении
    # L    - определяет величину отсечения пространства (L=8 или L=10)
        
    
    # меняем размер K - теперь K вектор
    if K is not np.array:
        K = np.array(K).reshape([len(K), 1])
    
    i = np.complex(0.0, 1.0) 
    x0 = np.log(S0 / K)   
    
    # усечённое пространство

    a = 0.0 - L * np.sqrt(tau)
    b = 0.0 + L * np.sqrt(tau)
    
    # Суммирование по k = 0 до k = N-1

    k = np.linspace(0, N-1, N).reshape([N, 1])  
    u = k * np.pi / (b - a);  

    # определяем коэффициенты и характеристическую функцию
    cf = ChFBatesModel(r,tau, par)
    H_k = CallPutCoefficients(CP,a,b,k)   
    mat = np.exp(i * np.outer((x0 - a) , u))
    temp = cf(u) * H_k 
    temp[0] = 0.5 * temp[0] 
    # суммируем полученные коэффициенты   
    value = np.exp(-r * tau) * K * np.real(mat.dot(temp))     
    return value


In [ ]:
# вычисление коэффициентов для соответствующего типа опционов
def CallPutCoefficients(CP,a,b,k):
    if CP==OptionType.CALL:                  
        c = 0.0
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        if a < b and b < 0.0:
            H_k = np.zeros([len(k),1])
        else:
            H_k = 2.0 / (b - a) * (Chi_k - Psi_k)  
    elif CP==OptionType.PUT:
        c = a
        d = 0.0
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k)               
    
    return H_k    

In [ ]:
# вспомогательная функция для вычисления коэффициентов в сумме
def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value

In [ ]:
# формула Блэка - Шоулса для стоимости опциона

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    
    K = np.array(K).reshape([len(K),1])
    sigma = np.array(sigma).reshape([len(sigma),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) 
    * tau) / (sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if CP == OptionType.CALL:
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif CP == OptionType.PUT:
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

In [ ]:
# вычисление подразумеваемой волатильности по цене опциона, страйку, времени до экспирации
# в качестве метода поиска корня используется метод Ньютона Рафсона
def ImpliedVolatility(CP,marketPrice,K,T,S_0,r):
    func = lambda sigma: np.power(BS_Call_Option_Price(CP,S_0,K,sigma,T,r) - marketPrice,1.0)
    impliedVol = optimize.newton(func, 0.7, tol=1e-5)
    return impliedVol

In [ ]:
# вычисление характеристической функции в модели Бейтса
def ChFBatesModel(r,tau,par):

    kappa = par[0]
    gamma = par[1]
    vbar = par[2]
    v0 = par[3]
    rho = par[4]
    xip = par[5]
    muj = par[6]
    sigmaj = par[7]


    i = np.complex(0.0,1.0)
    D1 = lambda u: np.sqrt(np.power(kappa-gamma*rho*i*u,2)+(u*u+i*u)*gamma*gamma)
    g  = lambda u: (kappa-gamma*rho*i*u-D1(u))/(kappa-gamma*rho*i*u+D1(u))
    C  = lambda u: (1.0-np.exp(-D1(u)*tau))/(gamma*gamma*(1.0-g(u)*D1(u)*tau)) \
        *(kappa-gamma*rho*i*u-D1(u))

    # тут нет множителя -r*tau, так как дисконтирование происходит в функции для COS метода

    Aheston  = lambda u: r * i*u *tau + kappa*vbar*tau/gamma/gamma *(kappa-gamma*rho*i*u-D1(u))\
        - 2*kappa*vbar/gamma/gamma*np.log((1-g(u)*np.exp(-D1(u)*tau))/(1-g(u)))
    
    A = lambda u: Aheston(u) - xip * i * u * tau *(np.exp(muj+0.5*sigmaj*sigmaj) - 1.0) + \
            xip * tau * (np.exp(i*u*muj - 0.5 * sigmaj * sigmaj * u * u) - 1.0)
       

    cf = lambda u: np.exp(A(u) + C(u)*v0)
    return cf 

In [ ]:
# моделирование двухмерной коррелированной модели Бейтса без модификации, предложенной в магистерской, на основе моделирования методом Эйлера
# возвращает 2 путя
def Bates2path(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12):
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj)
  
  dt = T / float(NoOfSteps)
  Z1 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z2 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z3 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z4 = np.random.normal(0.0,1.0,[1, NoOfSteps])

  # Моделирование пуассоновских процессов и их амплитуд
  J1 = np.random.normal(par1[6],par1[7],[1,NoOfSteps])
  J2 = np.random.normal(par2[6],par2[7],[1,NoOfSteps])
  Pois1 = np.random.poisson(par1[5]*dt,[1,NoOfSteps])
  Pois2 = np.random.poisson(par2[5]*dt,[1,NoOfSteps])

  W1 = np.zeros([1, NoOfSteps + 1])
  W2 = np.zeros([1, NoOfSteps + 1])
  W3 = np.zeros([1, NoOfSteps + 1])
  W4 = np.zeros([1, NoOfSteps + 1])
  V1 = np.zeros([1, NoOfSteps + 1])
  V2 = np.zeros([1, NoOfSteps + 1])
  X1 = np.zeros([1, NoOfSteps + 1])
  X2 = np.zeros([1, NoOfSteps + 1])
  V1[:,0] = par1[3]
  V2[:,0] = par2[3]
  X1[:,0] = np.log(S_01)
  X2[:,0] = np.log(S_02)
  time = np.zeros([1, NoOfSteps+1])    
  EeJ1 = np.exp(par1[6] + 0.5*par1[7]*par1[7])
  EeJ2 = np.exp(par2[6] + 0.5*par2[7]*par2[7])

  for i in range(0,NoOfSteps):
       
        Z2[:,i] = par1[4] * Z1[:,i] + np.sqrt(1.0-par1[4]**2) * Z2[:,i]
        Z4[:,i] = par2[4] * rho12 * Z1[:,i] + par2[4] * np.sqrt(1.0-rho12**2) * Z3[:,i] + np.sqrt(1.0-par2[4] ** 2) * Z4[:,i]
        Z3[:,i] = rho12 * Z1[:,i] + np.sqrt(1.0-rho12**2) * Z3[:,i]

        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        W3[:,i+1] = W3[:,i] + np.power(dt, 0.5)*Z3[:,i]
        W4[:,i+1] = W4[:,i] + np.power(dt, 0.5)*Z4[:,i]
        
        
        V1[:,i+1] = V1[:,i] + par1[0] * (par1[2] - V1[:,i]) * dt + par1[1] * np.sqrt(V1[:,i]) * (W2[:,i+1]-W2[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1],0.0)
        X1[:,i+1] = X1[:,i] + (mu[0] - 0.5 * V1[:,i] - par1[5]*(EeJ1-1)) * dt + np.sqrt(V1[:,i]) * (W1[:,i+1]-W1[:,i]) + J1[:,i] * Pois1[:,i]
        V2[:,i+1] = V2[:,i] + par2[0] * (par2[2] - V2[:,i]) * dt + par2[1] * np.sqrt(V2[:,i]) * (W4[:,i+1]-W4[:,i])
        V2[:,i+1] = np.maximum(V2[:,i+1],0.0)
        X2[:,i+1] = X2[:,i] + (mu[1] - 0.5 * V2[:,i] - par2[5]*(EeJ2-1)) * dt + np.sqrt(V2[:,i]) * (W3[:,i+1]-W3[:,i]) + J2[:,i] * Pois2[:,i]
        
        time[0][i+1] = time[0][i] + dt
  return np.exp(X1), np.exp(X2), time    


In [3]:
# функция для подсчёта выборочной корреляции между двумя путями
def EmpCorr(path1, path2):
  return np.corrcoef([np.log(path1[j + 1]/path1[j]) for j in range(len(path1) - 1)], [np.log(path2[j + 1]/path2[j]) for j in range(len(path2) - 1)])[0][1]

In [4]:
def check_pos_def(vec):
  for i in vec:
    if i <= 0:
      return False
  return True

In [5]:
def chol(M):
  L = np.zeros([len(M[0]), len(M[0])])
  for j in range(len(M[0])):
    for i in range(j, len(M[0])):
      if i == 0 and j == 0:
        L[i, j] = np.sqrt(M[i, j])
      if i == j and i != 0 and j != 0:
        if M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])) <= 0:
          L[i, j] = 10e-5
        else:
          L[i, j] = np.sqrt(M[i, j] - np.sum(np.array([L[i, k] ** 2 for k in range(j)])))
          
      else:
        L[i, j] = (M[i, j] - np.sum(np.array([L[j, k] * L[i, k ] for k in range(j)])))/L[j, j]
  return L

In [6]:
def chol_matr(par_model, corr_matr):
  C = np.zeros([len(par_model) * 2, len(par_model) * 2])
  
  for i in range(len(par_model)):
    C[i, i] = 1
    C[i, i + len(par_model)] = par_model[i][4]
    C[i + len(par_model), i ] = C[i, i + len(par_model)] 
  for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
    C[i, i] = 1
    for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
      
      C[i, j] = corr_matr[k1][k2]
      C[j, i] = C[i, j]
  
  w, v = np.linalg.eig(C)
  
  # проверяем положительную определённость, если не выполняется, то используем метод регуляризации Джэкеля
  if check_pos_def(w) == True:
   
    return np.linalg.cholesky(C)
  else:
    Lambda = np.zeros([len(w), len(w)])
    sp = np.zeros([len(w)])
    for i in range(len(w)):
      if w[i] >= 0:
        Lambda[i][i] = w[i]
        sp[i] = w[i] 
    T = np.zeros([len(w), len(w)])
    for i in range(len(w)):
      T[i][i] = 1/(np.power(v[i], 2).dot(sp))
    
    B = np.dot(np.dot(np.sqrt(T), v), np.sqrt(Lambda))
    Cnew = np.dot(B, np.transpose(B))
    w, v = np.linalg.eig(Cnew)
    for i in range(len(par_model) *2):
        Cnew[i, i] = 1
    return chol(Cnew)

In [7]:
# вычисление параметров для QE схемы

def FirstApproach(m, s2):
    b2 = 2.0 * m * m / s2 - 1.0 + np.sqrt(2.0 * m * m / s2)*np.sqrt((2.0 * m * m / s2) - 1.0)
    b  = np.sqrt(b2)
    a  = m  / (1.0 + b2)
    return a,b

def SecondApproach(m, s2):
    c = ((s2 / (m*m)) - 1.0) / ((s2 / (m*m)) + 1.0)
    d = (1.0 - c) / m
    return c, d

def CIRCDF(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    cdf =lambda x: st.ncx2.cdf(x/c,delta,kappaBar)
    return cdf

def CIRMean(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(- kappa * (t-s)))
    kappaBar = 4.0 * kappa * v_s * np.exp(-kappa * (t-s)) / (gamma * gamma * (1.0 - np.exp(-kappa*(t-s))))
    return c * (delta + kappaBar)

def CIRVar(kappa, gamma, vbar, s, t, v_s):
    delta = 4.0 *kappa*vbar/gamma/gamma
    c= 1.0/(4.0*kappa)*gamma*gamma*(1.0-np.exp(-kappa*(t-s)))
    kappaBar = 4.0*kappa*v_s*np.exp(-kappa*(t-s))/(gamma*gamma*(1.0-np.exp(-kappa*(t-s))))
    VarV = c*c*(2.0*delta+4.0*kappaBar)
    return VarV

In [8]:
# QE схема для моделирования  V(t)|V(s)
def QE_scheme(kappa, gamma, vbar, s, t, v_s, NoOfSamples):
    m  = CIRMean(kappa, gamma, vbar, s, t, v_s)
    s2 = CIRVar(kappa, gamma, vbar, s, t, v_s)
    if m < 0 or s2 < 0:
      print('параметры', kappa, gamma, vbar, s, t, v_s)
    aStar = 1.5 # параметр разделения
    if (s2/ (m * m) < aStar):

        # a и b - первый подход

        a, b = FirstApproach(m, s2)
        Z = np.random.normal(0.0, 1.0, [NoOfSamples, 1])
        
        if NoOfSamples > 1:
          Z = (Z - np.mean(Z)) / np.std(Z)
        
        A = a * np.power(b + Z, 2.0)

    else:

        # c & d - второй подход
        c, d = SecondApproach(m, s2)
        U = np.random.uniform(0.0, 1.0, [NoOfSamples, 1])
        A = 1.0 / d * np.log((1.0-c)/(1.0-U))
        A[U < c] = 0.0
    #print(A)
    return A  

In [9]:
# Моделирование многомерного пути активов в модели Бейтса без модификации
def multi_asset_path(par_model, corr_matrix, parS0, T, r, NoOfSteps):
   # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj) 
    dt = T / float(NoOfSteps)  
    V = np.zeros([len(parS0),  NoOfSteps+1])
    X = np.zeros([len(parS0),  NoOfSteps+1])
    xiEeJ = []
    for i in range(len(parS0)):
      xiEeJ.append(par_model[i][5] * (np.exp(par_model[i][6] + 0.5*par_model[i][7]*par_model[i][7]) - 1))
    
    Pois = np.array([np.random.poisson(par_model[i][5]*dt,[1,NoOfSteps + 1])[0] for i in range(len(parS0))])
    J = np.array([np.random.normal(par_model[i][6], par_model[i][7],[1 ,NoOfSteps + 1])[0] for i in range(len(parS0))])
    xiEeJ = np.array(xiEeJ)
    Vint = np.zeros([len(parS0),  NoOfSteps+1])
    f = np.zeros([len(parS0), 1])
    s = np.zeros([len(parS0), 1])
    L = chol_matr(par_model, corr_matrix)
    
    rho_m = np.zeros([len(parS0), len(parS0)])

    for i in range(len(parS0)):
      rho_m[i, i] = par_model[i][4]

    Ds = np.zeros([len(parS0), len(parS0)])
    for i in range(len(parS0)):
      V[i, 0] = par_model[i][3]
      X[i, 0] = np.log(parS0[i])
    L_star = np.zeros([len(parS0), len(parS0)])
    for k1, i in enumerate(range(len(par_model), len(par_model) * 2)):
      for k2, j in enumerate(range(len(par_model), len(par_model) * 2)):
          L_star[k1, k2] = L[i, j]
    time = np.zeros([NoOfSteps+1])
    
    for i in range(0,NoOfSteps):
        time[i+1] = time[i] + dt
        Z = np.random.normal(0.0, 1.0, [len(parS0), 1])
        for j in range(len(parS0)):
            #V[j,i+1] = CIR_Sample(NoOfPaths, kappa, gamma, vbar, 0, dt, V[:,i])
            V[j,i+1] = QE_scheme(par_model[j][0], par_model[j][1], par_model[j][2], time[i], time[i+1], V[j, i], 1)[0][0]
            Vint[j, i] = dt * (V[j,i+1] + V[j,i])/2
            f[j] = (V[j,i+1] - V[j,i] - par_model[j][0] * par_model[j][2] * dt + par_model[j][0] * Vint[j, i]) / par_model[j][1]
            Ds[j, j] = np.sqrt(Vint[j, i])
        
        
        X[:, i+1] = X[:, i] + (r  - Vint[:, i]/2 - xiEeJ) * dt + np.dot(rho_m , f).reshape([1, len(parS0)])[0] + np.dot(Ds, np.dot(L_star, Z)).reshape([1, len(parS0)])[0] + J[:, i] * Pois[:, i]
    return np.exp(X), time

In [10]:
def findind(arr, elem, par = 4):
  for i, j in enumerate(arr):
    if round(j, par) == elem:
      return i
  return False 

In [ ]:
# Вводится тикер
# Возвращает: 
# 1) Цена закрытия последнего торгового дня
# 2) Таблица цен опционов (строки соответствуют экспирации, а столбцы страйкам)
# 3) Экспирации приведённые к виду тау
# 4) Страйки
def option_data(ticker):
  ticker_dates = options.get_expiration_dates(ticker)
  ticker_dates = ticker_dates[-10:]
  calls = {}
  for i in ticker_dates:
    calls.update({'{0}'.format(i) : options.get_options_chain(ticker, i)['calls']})
  
  h_data = []
  for i in ticker_dates:
    h_data.append([i, calls[i]['Strike'], calls[i]['Last Price']])

  strikes = set.intersection(*[set(i[1]) for i in h_data])

  def findind(arr, elem, par = 4):
    for i, j in enumerate(arr):
      if round(j, par) == elem:
        return i
    return False 

  final_data = []

  for i in h_data:
    help_data = []
    for j in sorted(list(strikes)): 
      help_data.append(i[2][findind(i[1], j)])

    final_data.append(help_data)

  interval = '1d'
  period1 = int(time.mktime(datetime.datetime(2022, 4, 14, 23, 59).timetuple()))
  period2 = int(time.mktime(datetime.datetime.now().timetuple())) 
  query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
  df = pd.read_csv(query_string)
  return float(df[-1:]['Close']), np.array(final_data), np.array([(datetime.datetime.strptime(i, "%B %d, %Y") - datetime.datetime.now()).days/365 for i in ticker_dates]), np.array(sorted(list(strikes))).reshape([len(strikes),1])

In [ ]:
# калибровка модели Хестона
def Calibration_Bates(CP, S0, r, N, L, bounds, optimizer, market, expiration, strikes):
    # CP   - тип опциона
    # S0   - начальная цена базового актива
    # r    - процентная ставка (постоянная)
    # N    - число элементов в разложении
    # L    - определяет величину отсечения пространства (L=8 или L=10)
    # bounds - границы для параметров
    # optimizer - тип оптимизации : в данной работе применяется дифференциальная эволюция
    # market - данные, по которым калибруют (даны в виде подразумеваемых волатильностей)
    # expiration - вектор времён до экспирации
    # strikes - вектор страйков
    
    
    # функция ошибок 
    def Err_fun(x):
        # внутри суммирование идёт по страйкам при фиксированной экспирации, а далее идёт суммирвоание по экспирациям
        return np.sum(np.array([np.sum((CallPutOptionPriceCOSMthd(x, CP, S0, r, expiration[i], strikes, N, L).reshape([1,len(strikes)]) - np.array(market[i]).reshape([1,len(strikes)])) ** 2) for i in range(len(expiration))]))
    # оптимизация при заданных границах
    res = optimizer(Err_fun, bounds)
    return res.x, res.fun

In [ ]:
tickers = ['CVX']
CP  = OptionType.CALL
r   = 0.0028
N   = 500
L   = 12
bounds = [(0.5, 0.5), (0.01, 1.0), (0.01, 1.0), (0.01, 1.0), (-1.0, 1.0), (0.01, 10), (-2, 2), (0.01, 2)]
for i in tickers:
  a_file = open(f"{i}.pkl", "rb")
  data = pickle.load(a_file)
  a_file.close()
  par = Calibration_Bates(CP, data['S0'], r, N, L, bounds, optimize.differential_evolution, data['Option data'], data['Expirations'], data['Strikes'])
  print(i, par[0], par[1])
  data.update({'Parameters Bates': par[0]})
  b_file = open(f"{i}.pkl", "wb")
  pickle.dump(data, b_file)
  b_file.close()
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


CVX [ 0.5    0.634  0.241  0.154 -0.79   0.015 -0.425  0.265] 672.3794551071798


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_differentialevolution.py:1154: RuntimeWarning: invalid value encountered in true_divide
  return (parameters - self.__scale_arg1) / self.__scale_arg2 + 0.5


In [11]:
def empcorrmatr(tickers):
  num_tickers = {}
  num = []
  for ind, ticker in enumerate(tickers):
    num_tickers.update({f'{ind}': ticker})
    num.append(ind)
  empcor = np.zeros([len(num), len(num)])
  for i in range(len(num) - 1):
    b_file = open(f"{num_tickers[f'{i}']}HD.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    path1 = data[num_tickers[f'{i}']]
    for j in range(i + 1, len(num)):
      b_file = open(f"{num_tickers[f'{j}']}HD.pkl", "rb")
      data = pickle.load(b_file)
      path2 = data[num_tickers[f'{j}']]
      b_file.close()
      empcor[i, j] = EmpCorr(path1, path2)
      empcor[j, i] = empcor[i, j]
  for i in range(len(num)):
    empcor[i, i] = 1
  return empcor

In [ ]:
N1 = [10, 20, 30, 50, 75, 100]
r = 0.0028
alltickers = [['UNH', 'CVX', 'BA'], ['NVDA', 'AMD', 'QCOM']]
for j in N1:
  for i in alltickers:
    tickers = i
    print(corr_matr_X(tickers, r, j), tickers)


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


par1 [ 0.5    0.36   0.065  0.069 -0.59   0.122 -1.114  0.043]
par2 [ 0.5    0.215  0.076  0.13  -0.753  0.01  -0.189  0.054]
0.36399592609704123 1 0.46399592609704127
x[k] 0.6766080980154447 k 29 0.00011154324546198455
par1 [ 0.5    0.36   0.065  0.069 -0.59   0.122 -1.114  0.043]
par2 [ 0.5    0.195  0.129  0.131  0.004  0.229 -0.792  0.301]
0.3062512274386642 1 0.40625122743866415
x[k] 0.8471401009610616 k 46 0.0014278970828454586
par1 [ 0.5    0.215  0.076  0.13  -0.753  0.01  -0.189  0.054]
par2 [ 0.5    0.195  0.129  0.131  0.004  0.229 -0.792  0.301]
0.4458673464975015 1 0.5458673464975015
x[k] 0.7558059493039837 k 33 0.000464952765127855
(array([[1.   , 0.677, 0.847],
       [0.677, 1.   , 0.756],
       [0.847, 0.756, 1.   ]]), array([[1.   , 0.464, 0.406],
       [0.464, 1.   , 0.546],
       [0.406, 0.546, 1.   ]])) ['UNH', 'CVX', 'BA']
par1 [ 0.5    0.432  0.187  0.299 -0.842  0.205 -0.921  0.765]
par2 [ 0.5    0.659  0.317  0.29  -0.418  0.485 -0.605  0.372]
0.431069044980

In [12]:
# вычисление интенсивностей по введённым тикерам, и границам промежутка
# на выходе матрица совместных прыжков за указанный промежуток
!pip install yfinance
import yfinance as yf
def intensities(tickers, t1, t2):
  data = []
  for i in tickers:
    p1 = datetime.datetime(t1[2], t1[1], t1[0], 23, 59)
    p2 = datetime.datetime(t2[2], t2[1], t2[0], 23, 59)
    period1 = int(time.mktime(p1.timetuple()))
    period2 = int(time.mktime(p2.timetuple()))
    df = yf.download(i, p1 , p2)
    data.append(np.array(df[['High', 'Low']]))

  jum = np.zeros([len(data), len(data)])

  for i in range(len(data[0]) - 1):
    for j in range(len(data) - 1):
      for k in range(j + 1, len(data)):
        if data[j][i, 0] < data[j][i+1, 1] and data[k][i, 0] < data[k][i+1, 1]:
          jum[j][k] += 1
        elif data[j][i, 1] > data[j][i+1, 0] and data[k][i, 1] > data[k][i+1, 0]:
          jum[j][k] += 1
      
      if data[j][i, 0] < data[j][i+1, 1] or data[j][i, 1] > data[j][i+1, 0]:
        jum[j][j] += 1
    if data[len(data) - 1][i, 0] < data[len(data) - 1][i+1, 1] or data[len(data) - 1][i, 1] > data[len(data) - 1][i+1, 0]:
        jum[len(data) - 1][len(data) - 1] += 1
    
  voc = {}
  for l, m in enumerate(tickers):
    voc.update({m : jum[l]})

  b_file = open("Jumps.pkl", "wb")  
  pickle.dump(voc, b_file)
  b_file.close()
  return jum

In [ ]:
# функция калибровки параметра theta в реальной мере в модели Бейтса
# на входе параметры модели Бейтса в риск-нейтральной мере, y - лист дневного приращения логарифмических цен на актив
# к сожалению после попыток оптимизации кода значения функции всё равно вычисляются очень долго
def RWCBates(par, y, r):
  
  def fun(x):
    kappa = par[0]
    omega = par[1]
    rho = par[4]
    lamb = par[5]
    alpha = par[6]
    beta = par[7]
    eta = x
    mu = r + kappa * (eta - par[2])/(rho * omega)
    tau = 1/252
    i   = np.complex(0.0,1.0)
    a0 = 2 * kappa * eta / (omega * omega)
    b0 = (omega * omega) / (2 * kappa)
    def kG(u):
      return kappa - i * u * rho * omega
    def psiX(u):
      return lamb * (np.exp(i * beta * u - (1/2) * np.power(alpha, 2) * np.power(u, 2)) - 1) - i * u * lamb * (np.exp(beta + (1/2) * np.power(alpha, 2)) - 1)
    def psi(u):
      return -0.5 * (u * u + i * u)
    def gamma(u):
      return np.sqrt(np.power(kG(u), 2) - 2 * (omega * omega) * psi(u))
    def A(u):
      return (kG(u) - gamma(u))/(kG(u) + gamma(u))
    def C(u):
      return i * u * mu * tau + (kappa * eta / (omega * omega)) * (kG(u) - gamma(u)) * tau - (2 * kappa * eta / (omega * omega)) * np.log((A(u) * np.exp(- gamma(u) * tau ) - 1)/(A(u) - 1)) + psiX(u) * tau
    def D(u):
      return ((kG(u) - gamma(u))/ (omega * omega)) * ((1-np.exp(-gamma(u) * tau))/(1 - A(u) * np.exp(-gamma(u) * tau)))
    def F0(u, a, b):
      return np.exp(C(u)) / np.power(1 - b * D(u), a)
    def f(u, a, b):
      return C(u) - a * np.log(1 - b * D(u))
    def H(u):
      return (kG(u) + gamma(u)) / (1 - A(u) * np.exp(-gamma(u) * tau))
    def K(u):
      return (omega * omega) * (1 - np.exp(-gamma(u) * tau)) / H(u)
    def Cv(u):
      return ((2 * kappa * eta / (omega * omega)) * K(u) * i)
    def Cvv(u):
      return -1 * (2 * kappa * eta / (omega * omega)) * np.power(K(u), 2) 
    def R(u):
      return 4 * np.exp(-gamma(u) * tau) * np.power(gamma(u)/H(u) , 2)
    def Dv(u):
      return R(u) * i
    def Dvv(u):
      return -2 * K(u) * R(u)
    def fv(u, a, b):
      return Cv(u) + (a * b * Dv(u))/(1 - b * D(u))
    def fvv(u, a, b):
      return Cvv(u) + (a * b * Dv(u) + a * b * b * (np.power(Dv(u), 2) - D(u) * Dvv(u))) / np.power( 1 - b * D(u), 2)
    def F(u,a, b):
      return np.exp(f(u, a, b))
    def Fv(u, a, b):
      return fv(u, a, b) * F(u, a, b)
    def Fvv(u,a, b):
      return (fvv(u, a, b) + np.power(fv(u, a, b), 2)) * F(u, a, b)

    def find_min(x1, x2):
      for e1, e2 in enumerate(x1):
        if e2 > x2:
          return e1 - 3, e1 + 3

    h = []
    N = np.power(2, 12)
    du = 0.25
    dx = 8 * np.pi / N
    b = np.pi * 4
    x = np.array([ -b + (t - 1) * dx for t in range(1 , N + 1)])
    w = np.array([1.0 for t in range(N)])
    w[0] = 1/2
    w[N-1] = 1/2


    u = [(j -1 )* du for j in range(1, N+1)]
    
    py = []
    # 3000
    for n in tqdm(range(len(y) - 1)):
      h0 = []
      h1 = []
      h2 = []
      h3 = []
      bou = find_min(x, y[n + 1])
      
      if n == 0:
        
        F0s = [np.exp(i* b * u[j -1] ) * F0(u[j -1], a0, b0) * w[j -1] * du for j in range(1, N +1)]

        

        for k in range(bou[0], bou[1]):
          
          
          
          h0.append(np.real(np.sum([np.exp(-i * 2 * np.pi * (k - 1) * (j - 1)/N ) * F0s[j -1]  for j in range(1, N +1)])))

        h0 = np.array(h0)
        
        S0 = CubicSpline(x[bou[0] : bou[1]], h0)
        py.append(S0(y[n + 1]))
      else:

        Fs = [np.exp(i* b * u[j -1] ) * F(u[j -1], a0, b0) * w[j -1] * du for j in range(1, N +1)]
        fvs = [fv(u[j -1], a0, b0) for j in range(1, N +1)]
        
        Fvs = [fvs[j-1] * Fs[j -1] for j in range(1, N +1)]
        Fvvs = [(fvv(u[j -1], a0, b0) + np.power(fvs[j-1], 2)) * Fs[j -1] for j in range(1, N +1)]
        
        for k in range(bou[0], bou[1]):
          
          
          h1.append(np.real(np.sum([np.exp(-i * 2 * np.pi * (k - 1) * (j - 1)/N ) *  Fs[j-1]   for j in range(1, N +1)]))/np.pi)
          h2.append(np.imag(np.sum([np.exp(-i * 2 * np.pi * (k - 1) * (j - 1)/N ) *  Fvs[j-1]   for j in range(1, N +1)]))/np.pi)
          h3.append(-np.real(np.sum([np.exp(-i * 2 * np.pi * (k - 1) * (j - 1)/N ) * Fvvs[j-1]   for j in range(1, N +1)]))/np.pi)
        h1 = np.array(h1)
        S1 = CubicSpline(x[bou[0] : bou[1]], h1)
        py.append(S1(y[n + 1]))
        h2 = np.array(h2)
        S2 = CubicSpline(x[bou[0] : bou[1]], h2)
        m1 = S2(y[n + 1])/py[-1]
        h3 = np.array(h3)
        S3 = CubicSpline(x[bou[0] : bou[1]], h3)
        m2 = S3(y[n + 1])/py[-1]
        b0 = (m2 - np.power(m1, 2))/m1
        a0 = m1 /b0
    
    return -np.sum(np.log(np.array(py)))
  
  
  #res = optimize.differential_evolution(err_fun, bounds, atol = 1e-1)
  res = optimize.differential_evolution(fun, bounds=[(0.01, 1)])
  return res.x[0]
  
  
  
 

In [ ]:
# оценка параметра theta в модели Бейтса в случае, если калибровка была бы быстрее 
def par_teta_est2(ticker, r):
  b_file = open(f"{ticker}HD.pkl", "rb")
  data = pickle.load(b_file)
  b_file.close()
  v = np.array(data[ticker]) 
  y = np.array([np.log(v[j + 1]/v[j]) for j in range(len(v) - 1)])
  print(y)
  S0 = v[0]
  b_file = open(f"{ticker}.pkl", "rb")
  data = pickle.load(b_file)
  b_file.close()
  print(data['Parameters Bates'])
  teta = RWCBates(data['Parameters Bates'] , y, r)
  return teta, S0 , data['Time range']

In [13]:
# Возвращает оценённый параметр vbar, цену начала периода!!!! и длину интервала
def par_teta_est(ticker):
  b_file = open(f"{ticker}HD.pkl", "rb")
  data = pickle.load(b_file)
  b_file.close()
  v = np.array(data[ticker]) 
  S0 = v[0]
  return np.sum(np.array([np.log(v[j + 1]/v[j]) for j in range(len(v) - 1)]) ** 2) / data['Time range'], S0 , data['Time range']

In [14]:
def Janckel_reg(M):
  w, v = np.linalg.eig(M)
  if check_pos_def(w) == True:
    return M
  else:
    Lambda = np.zeros([len(w), len(w)])
    sp = np.zeros([len(w)])
    for i in range(len(w)):
      if w[i] >= 0:
        Lambda[i][i] = w[i]
        sp[i] = w[i] 
    T = np.zeros([len(w), len(w)])
    for i in range(len(w)):
      T[i][i] = 1/(np.power(v[i], 2).dot(sp))
    
    B = np.dot(np.dot(np.sqrt(T), v), np.sqrt(Lambda))
    Cnew = np.dot(B, np.transpose(B))
    return Cnew 

In [15]:
# генерация путей двухмерной модели Бейтса с модификацией введённой в работе
# на вход дополнительно подаются оценки вклада сектора в интенсивность прыжков 
# !!!! на входе подаются уже отсортированные параметры
def Bates2pathmod(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens):
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho, 6=xip, 7=muj, 8=sigmaj)
  
  dt = T / float(NoOfSteps)
  Z1 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z2 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z3 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  Z4 = np.random.normal(0.0,1.0,[1, NoOfSteps])
  xi_star = []
  xi_bar = []
  xi_star.append(par1[5] * intens[0])
  xi_bar.append(par1[5] - xi_star[0])
  for i in range(1, 1):
    xi_star.append(par2[5] * intens[i] - sum(xi_star))
    xi_bar.append(par2[5] - sum(xi_star))

  xi_bar.append(par2[5] - sum(xi_star))

  Pois1 = np.array([np.random.poisson(xi_star[i] *dt,[1,NoOfSteps + 1])[0] for i in range(1)])
  Pois2 = np.array([np.random.poisson(xi_bar[i]*dt,[1,NoOfSteps + 1])[0] for i in range(2)])
  Pois = np.array([Pois2[i] for i in range(2)])
  for i in range(2):
    for j in range(i + 1):
      if j == 1:
        break
      else:
        Pois[i] += Pois1[j]
  
  Cj = np.zeros([2,  2])
  for i in range(1):
    for j in range(i + 1, 2):
      Cj[i, j] = min(intens[i], intens[j]) 
      Cj[j, i] = Cj[i, j]
    Cj[i, i] = 1

  Cj[1, 1] = 1
  Cj = Janckel_reg(Cj)
  Cj[0, 1] = Cj[0, 1] * par1[7]* par2[7]
  Cj[1, 0] = Cj[0, 1]
  Cj[0, 0] = par1[7]* par1[7]
  Cj[1, 1] = par2[7] * par2[7]

  Lj = chol(Cj)

  mu1 = par1[6]
  mu2 = par2[6]
  # Моделирование пуассоновских процессов и их амплитуд

  W1 = np.zeros([1, NoOfSteps + 1])
  W2 = np.zeros([1, NoOfSteps + 1])
  W3 = np.zeros([1, NoOfSteps + 1])
  W4 = np.zeros([1, NoOfSteps + 1])
  V1 = np.zeros([1, NoOfSteps + 1])
  V2 = np.zeros([1, NoOfSteps + 1])
  X1 = np.zeros([1, NoOfSteps + 1])
  X2 = np.zeros([1, NoOfSteps + 1])
  V1[:,0] = par1[3]
  V2[:,0] = par2[3]
  X1[:,0] = np.log(S_01)
  X2[:,0] = np.log(S_02)
  time = np.zeros([1, NoOfSteps+1])    
  EeJ1 = np.exp(par1[6] + 0.5*par1[7]*par1[7])
  EeJ2 = np.exp(par2[6] + 0.5*par2[7]*par2[7])

  for i in range(0,NoOfSteps):
       
        Z2[:,i] = par1[4] * Z1[:,i] + np.sqrt(1.0-par1[4]**2) * Z2[:,i]
        Z4[:,i] = par2[4] * rho12 * Z1[:,i] + par2[4] * np.sqrt(1.0-rho12**2) * Z3[:,i] + np.sqrt(1.0-par2[4] ** 2) * Z4[:,i]
        Z3[:,i] = rho12 * Z1[:,i] + np.sqrt(1.0-rho12**2) * Z3[:,i]

        W1[:,i+1] = W1[:,i] + np.power(dt, 0.5)*Z1[:,i]
        W2[:,i+1] = W2[:,i] + np.power(dt, 0.5)*Z2[:,i]
        W3[:,i+1] = W3[:,i] + np.power(dt, 0.5)*Z3[:,i]
        W4[:,i+1] = W4[:,i] + np.power(dt, 0.5)*Z4[:,i]
        Zj1 = np.random.normal(0.0, 1.0, [1, 1])
        Zj2 = np.random.normal(0.0, 1.0, [1, 1])
        V1[:,i+1] = V1[:,i] + par1[0] * (par1[2] - V1[:,i]) * dt + par1[1] * np.sqrt(V1[:,i]) * (W2[:,i+1]-W2[:,i])
        V1[:,i+1] = np.maximum(V1[:,i+1],0.0)
        X1[:,i+1] = X1[:,i] + (mu[0] - 0.5 * V1[:,i] - par1[5]*(EeJ1-1)) * dt + np.sqrt(V1[:,i]) * (W1[:,i+1]-W1[:,i]) + (mu1 + Zj1 * Lj[0][0]) * Pois[0][i]
        V2[:,i+1] = V2[:,i] + par2[0] * (par2[2] - V2[:,i]) * dt + par2[1] * np.sqrt(V2[:,i]) * (W4[:,i+1]-W4[:,i])
        V2[:,i+1] = np.maximum(V2[:,i+1],0.0)
        X2[:,i+1] = X2[:,i] + (mu[1] - 0.5 * V2[:,i] - par2[5]*(EeJ2-1)) * dt + np.sqrt(V2[:,i]) * (W3[:,i+1]-W3[:,i]) + (mu2 + Zj1 * Lj[1][0] + Zj2 * Lj[1][1]) * Pois[1][i]
        
        time[0][i+1] = time[0][i] + dt
  return np.exp(X1), np.exp(X2), time    

In [16]:
def corrBatescalibration(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho_emp, bounds, intens):
  def err_fun(x):
    return (E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, x, intens) - rho_emp) ** 2
  #res = optimize.minimize(fun=err_fun, x0=0, method='Nelder-Mead', bounds=bounds, tol = 1e-2)
  x = np.linspace(rho_emp - 0.1, 1, 60)
  k = 0
  zf = 10
  # с учётом того, что данные по эмпирическим корреляциям есть, то было решено взять максимально простой метод поиск минимума (по сути перебор) и не брать метод дифференциальной эволюции, 
  #так как важно получить на выходе положителньо определённую матрицу и не потратить много времени
  # решением задачи минимизации оказывается первое число, на котором ошибка оказывается меньше 0.001, 
  # то есть расхождение ожидаемой корреляции от эмпирической не больше 0,03
  for j, i in enumerate(x):
    y = err_fun(x[j])
    if y <= 0.001:
      k =  j 
      zf = y
      break
    if zf > y:
      k = j
      zf = y
  return x[k]


In [17]:
# функция для подсчёта ожидаемой выборочной корреляции в модели Бейтса
# генерация путей происходит с использование модификации
def E_rho_emp(N, par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens):
  s = 0
  for i in range(N):
    a = Bates2pathmod(par1, par2, NoOfSteps, T, mu, S_01, S_02, rho12, intens)
    s += EmpCorr(a[0][0], a[1][0])
  return s / N

In [18]:
def corr_calibration(par_model, par_S_0, N, NoOfSteps, T, mu, rho_emp_matr, intens):
  # par_model - матрица, строка i которой состоит из откалиброванных параметров модели Хестона для актива i
  # par_S_0 - вектор начальных стоимостей активов
  # rho_emp_matr - матрица исторических корреляций
  #intens - лист оценённых вкладов сектора в интенсивность прыжков
  # !!! на входе подаются отсортированные параметры
  bounds = [(-1.0, 1.0)]
  corr_Xi_Xj = np.zeros([len(par_S_0), len(par_S_0)])
  # вычисляем матрицу мгновенных корреляций между активами
  for i in range(len(par_S_0)):
    corr_Xi_Xj[i][i] = 1
    for j in range(i + 1, len(par_S_0)):
      corr_Xi_Xj[i][j] = corrBatescalibration(N, par_model[i], par_model[j], NoOfSteps, T, np.array([mu[i], mu[j]]), par_S_0[i], par_S_0[j], rho_emp_matr[i][j], [(-1, 1)], [intens[i], intens[j]])
      corr_Xi_Xj[j][i] = corr_Xi_Xj[i][j]
  return corr_Xi_Xj

In [19]:
# вычисление откалиброванной матрицы по введённым тикерам и листу соответствующих им вкладов сектора в интенсивность прыжков
def corr_matr_X(tickers, r, N, intens):
  v_P = []
  S_0 = []  
  for ticker in tickers:
    k = par_teta_est(ticker)
    S_0.append(k[1])
    v_P.append(k[0])
    T = k[2]
  mu_P = []
  Par = []
  # параметры (1=kappa, 2=gamma, 3=vbar, 4=v0, 5=rho)
  for ind, ticker in enumerate(tickers):
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    data['Parameters Bates'][2] = v_P[ind]
    Par.append(data['Parameters Bates'])
    d = data['Parameters Bates']
    mu_P.append(r + d[0] * (v_P[ind]-d[2]) / (d[4] * d[1]))
  empcorr = empcorrmatr(tickers)
  NoOfSteps = int(T * 252)
  return corr_calibration(Par, S_0, N, NoOfSteps, T, mu_P, empcorr, intens) , empcorr
  

In [20]:
# сортировка тикеров 
# На выходе лист отсортированных тикером и соответствующий лист вкладов сектора в интенсивность прыжков
def sort_tickers(tickers):
  voc = {}
  b_file = open("Intensities.pkl", "rb")  
  jum = pickle.load(b_file)
  b_file.close()
  for i in tickers:
    b_file = open(f"{i}.pkl", "rb")  
    dt = pickle.load(b_file)
    b_file.close()
    voc.update({i: dt['Parameters Bates'][5] * jum[i]})
  sorted_values = sorted(voc.values()) 
  final_int = []
  sorted_tickers = []
  for i in sorted_values:
      for k in voc.keys():
          if voc[k] == i:
              sorted_tickers.append(k)
              final_int.append(jum[k])
              break
              
  return sorted_tickers, final_int

In [21]:
#  по введённым тикерам возвращает параметры откалиброванных моделей 
# и начальную стоимость в нужном формате, чтобы потом их отправлять на вход функции autocallbondprice
def par_data_final(tickers):
  Par = []
  S0 = []
  for ticker in tickers:
    b_file = open(f"{ticker}.pkl", "rb")
    data = pickle.load(b_file)
    b_file.close()
    Par.append(data['Parameters Bates'])
    b_file = open(f"{ticker}HD.pkl", "rb")
    data = pickle.load(b_file)
    S0.append(np.array(data[ticker])[-1:])
    b_file.close()
  return Par, S0

In [22]:
def autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N, dates, coupon, barier, autocall, nom, ins):
  price = []
  count = 0
  for i in range(N):
    C = 0
    memory = 0
    paths = multi_asset_path(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps)
    for k in range(len(par_S_0)):
      paths[0][k, :] = paths[0][k, :] / par_S_0[k]
    for l, j in enumerate(dates):
        d = min(paths[0][:, findind(paths[1], j)])
        if d < barier and l == len(dates) - 1:
          C +=  np.exp(- r * j) * d * nom  / barier
          count += 1
        elif barier <= d <= autocall and l == len(dates) - 1:
          memory += 1
          C += np.exp(- r * j) * (memory * nom * coupon / 100 + (1 - ins) * nom)
        elif d > autocall and l == 0:
          C = np.exp(- r * j) * (2 * nom * coupon / 100 + nom)
          break
        elif d > autocall:
          memory += 1
          C += np.exp(- r * j) * (memory * nom * coupon / 100 + nom)
          break
        elif d < barier:
          memory += 1
        elif barier <= d <= autocall:
          memory += 1
          C += np.exp(- r * j) * memory * nom * coupon / 100
          memory = 0
    price.append(C)
  price = np.array(price)
  z = 1.96
  m = np.mean(price)
  s = np.std(price, ddof=1)
  return m  , round(count / N, 3), m - z * s / np.sqrt(N) , m + z * s / np.sqrt(N)

In [ ]:
N1 = 75
r = 0.0028

N2 = 10000


coupon = 4.5
barier = 0.65
autocall = 1
nom = 1000
ins = 0.01



#assets = [['CVX'], ['UNH'], ['BA'], ['NVDA'], ['QCOM'], ['AMD'], ['NVDA', 'QCOM'], ['BA', 'CVX'], ['NVDA', 'AMD', 'QCOM'], ['UNH', 'CVX', 'BA']]
assets = [['NVDA', 'AMD'], ['QCOM', 'AMD'] , ['BA', 'UNH'] ]
Expirations = [1, 2 , 3, 5, 10]


for j in assets:
    res = []
    tickers = sort_tickers(j)
    C = corr_matr_X(tickers[0], r, N1, tickers[1])
    print(C)
    P = par_data_final(tickers[0])
    par_model = P[0]
    rho_emp_matr = C[0]
    par_S_0 = P[1]
    b_file = open("results_Bates.pkl", "rb")
    res_voc = pickle.load(b_file)
    b_file.close()
    for i in Expirations:
        T = i
        NoOfSteps =  T * 252
        dates = [k/4 for k in range(1, T * 4 + 1)]
      
        results = autocallbondprice(par_model, rho_emp_matr, par_S_0, T, r, NoOfSteps, N2, dates, coupon, barier, autocall, nom, ins)
        res.append([i, results[0], results[1], results[2], results[3]])
        print(j, [i, results[0], results[1], results[2], results[3]])
    res_voc.update({"".join(j): res})
    b_file = open("results_Bates.pkl", "wb")
    pickle.dump(res_voc, b_file)
    b_file.close()



(array([[1.   , 0.788],
       [0.788, 1.   ]]), array([[1.   , 0.531],
       [0.531, 1.   ]]))
['NVDA', 'AMD'] [1, 948.1708806964187, 0.31, 942.558706102053, 953.7830552907845]
['NVDA', 'AMD'] [2, 945.0012684840494, 0.305, 938.3589858758453, 951.6435510922536]
['NVDA', 'AMD'] [3, 948.4607656100668, 0.289, 941.1196564457751, 955.8018747743585]
['NVDA', 'AMD'] [5, 964.8086569085052, 0.276, 956.6763566401289, 972.9409571768815]
['NVDA', 'AMD'] [10, 1009.7124910574224, 0.258, 999.7264234404545, 1019.6985586743904]
(array([[1.   , 0.596],
       [0.596, 1.   ]]), array([[1.   , 0.399],
       [0.399, 1.   ]]))
['QCOM', 'AMD'] [1, 985.4408953993328, 0.292, 980.6718937856521, 990.2098970130136]
['QCOM', 'AMD'] [2, 986.6597342441794, 0.296, 980.6961763104545, 992.6232921779043]
['QCOM', 'AMD'] [3, 989.6741776105829, 0.297, 982.9170059753656, 996.4313492458002]
['QCOM', 'AMD'] [5, 1012.3014525959022, 0.28, 1004.4456978017785, 1020.157207390026]
['QCOM', 'AMD'] [10, 1069.4935866607457, 0.251, 

In [3]:
b_file = open("results_Bates.pkl", "rb")  
jum = pickle.load(b_file)
b_file.close()

In [10]:
b_file = open("results_Bates.pkl", "wb")
pickle.dump(jum, b_file)
b_file.close()

In [11]:
b_file = open("results_Bates.pkl", "rb")  
lo = pickle.load(b_file)
b_file.close()
lo

{'AMD': [[1,
   1022.2030045891509,
   0.171,
   1018.0679338100622,
   1026.3380753682395],
  [2, 1033.2436031584218, 0.159, 1028.428547358435, 1038.0586589584086],
  [3, 1039.4745914515208, 0.149, 1034.18515500197, 1044.7640279010716],
  [5, 1062.76827161716, 0.135, 1056.879120860307, 1068.657422374013],
  [10, 1093.5932891277862, 0.117, 1086.14491699486, 1101.0416612607123]],
 'BA': [[1, 1059.4691300874176, 0.121, 1056.428272843615, 1062.5099873312201],
  [2, 1073.785844168761, 0.123, 1069.9827841590682, 1077.588904178454],
  [3, 1078.1419415664282, 0.127, 1073.6639209429807, 1082.6199621898757],
  [5, 1094.7663057571858, 0.122, 1089.4523174153487, 1100.080294099023],
  [10, 1125.5776487498338, 0.107, 1118.4323147533607, 1132.7229827463068]],
 'BACVX': [[1,
   1046.4477305413918,
   0.194,
   1042.952337289113,
   1049.9431237936706],
  [2, 1055.846598291874, 0.216, 1051.2803787885891, 1060.4128177951588],
  [3, 1066.173385492572, 0.22, 1060.8015123518783, 1071.5452586332656],
  [5,